In [31]:
import osmnx as ox
from pyproj import Proj

In [24]:
ox.__version__

'1.9.4'

In [11]:
place = "ACT, Australia"


In [10]:
# t = ox.geocode_to_gdf("ACT, Australia", which_result=None)
# type(t['geometry'].iloc[0])

shapely.geometry.polygon.Polygon

In [25]:
ox.config(use_cache=True, log_console=True)
G = ox.graph_from_place(place, network_type='bike', simplify=True)

C:\Temp\ipykernel_6332\3010082534.py:1: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.config(use_cache=True, log_console=True)


In [26]:
# get a GeoSeries of consolidated intersections
G_proj = ox.project_graph(G)
G2 = ox.consolidate_intersections(G_proj)


In [29]:
Gp = ox.project_graph(G2)

In [43]:
orig = [149.038124, -35.233842]
dest = [149.084816, -35.252348]
# Set up your UTM projection for ACT (Zone 55S)
proj_utm = Proj(proj='utm', zone=55, south=True, ellps='WGS84')
x1, y1 = proj_utm(orig[0], orig[1])
x2, y2 = proj_utm(dest[0], dest[1])

orig_node = ox.nearest_nodes(Gp, x1, y1)
dest_node = ox.nearest_nodes(Gp, x2, y2)
print(orig_node)
print(dest_node)

9579
6444


In [44]:
# find the shortest path (by distance) between these nodes then plot it
routes = ox.k_shortest_paths(G2, orig_node, dest_node, k=3, weight="length")
route_list = list(routes)


In [46]:
route_coords=[]
for i, route in enumerate(route_list):
    route_gdf = ox.routing.route_to_gdf(G2, route)
    route_length = route_gdf["length"].sum()
    print(f"Length of route {i+1}: {route_length} meters")

    # Extract the coordinates of the route
    coords = [(point.xy[1][0], point.xy[0][0]) for point in route_gdf.geometry]  # (lat, lon) format
    
    route_coords.append(coords)
print(route_coords)

Length of route 1: 5641.504069066369 meters
Length of route 2: 5643.212014488999 meters
Length of route 3: 5644.702914817996 meters
[[(6099160.463292609, 685425.975529575), (6099129.893893488, 685353.4779063907), (6099009.457275188, 685265.3034023275), (6098766.570395689, 685218.4773676815), (6098720.291735907, 685232.412421319), (6098657.268234268, 685294.0241892751), (6098634.606778711, 685326.5354406331), (6098597.566227471, 685483.7727386802), (6098497.377233095, 685616.322066714), (6098477.3612192925, 685620.2170225739), (6098350.557317756, 685634.9269508591), (6098327.180693739, 685710.497589728), (6098246.49876664, 685958.6039247421), (6098091.543334673, 686314.2261968546), (6098045.8953872165, 686600.9405710574), (6098023.981825297, 686873.03493985), (6098006.941190542, 687028.0734791672), (6098001.171826271, 687093.0379591824), (6097983.901329768, 687240.2270285134), (6097868.005024812, 687495.170244976), (6097734.577424798, 687690.9147645261), (6097679.753535016, 687789.65256

In [ ]:
for i, route in enumerate(routes):
    folium.PolyLine(route, color=["red", "blue", "green"][i], weight=5, opacity=0.7, popup=f'Route {i+1}').add_to(m)

# Add markers for the origin and destination
folium.Marker(location=st.session_state.start_coords, popup='Origin').add_to(m)
folium.Marker(location=st.session_state.end_coords, popup='Destination').add_to(m)
